In [1]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from deap_preprocessor import DEAPDataProcessor
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

In [2]:
processor = DEAPDataProcessor(data_folder="C://EEG_Emotion_Project/DEAP_code/DEAP/data_preprocessed_python/")
processor.load_data()
processor.split_channels()
# print(processor.labels.shape)
labels_encoded = processor.get_label_dataframe().values
print(f"标签的数据大小为{labels_encoded.shape}")
eeg_data = processor.get_eeg_data()
print(f"总数据的大小为{eeg_data.shape}")

eeg_data = eeg_data[:, :, 4224:]
print(type(eeg_data))

# # 转换为 numpy 数组
# eeg_data = np.array(eeg_data) # 
# labels_encoded = np.array(labels_encoded)
# print(f"分割后的总训练数据大小为: {eeg_data.shape}")
# print(f"分割后的总训练标签大小为: {labels_encoded.shape}")
# 
# scaler = StandardScaler()
# eeg_data = scaler.fit_transform(eeg_data.reshape(-1, eeg_data.shape[-1])).reshape(eeg_data.shape)

In [3]:
def compute_DE(signal):
    variance = np.var(signal, ddof=1)  
    return math.log(2 * math.pi * math.e * variance) / 2  # 微分熵求取公式

In [4]:
eeg_DE_data = []
eeg_DE_data_per_subject = []
for data in eeg_data:
    for signal in data:
        DE_data = compute_DE(signal)
        eeg_DE_data_per_subject.append(DE_data)
    eeg_DE_data.append(eeg_DE_data_per_subject)
    eeg_DE_data_per_subject = []

eeg_DE_data = np.array(eeg_DE_data)
print(eeg_DE_data.shape)

In [5]:
label = []
for i in labels_encoded:
    label.append(i[0])  # 收集 0：愉悦度； 1：唤醒度
    
label = np.array(label)
print(label.shape)

In [7]:
data = []
for i in eeg_DE_data:
    data.append(i.flatten())

data = np.array(data)
print(data.shape)

In [8]:
dataset = pd.DataFrame(data)
dataset["label"] = label
dataset.shape

In [9]:
x=dataset.iloc[:,:-1] 
y=dataset.iloc[:,-1]

In [10]:

# Splitting the data into training and testing sets 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20) 


In [11]:
# Defining the parameters grid for GridSearchCV 
param_grid={'C':[0.1,1,10,100], 
            'gamma':[0.0001,0.001,0.1,1], 
            'kernel':['rbf','poly']} 
  
# Creating a support vector classifier 
svc = SVC(probability=True) 
  
# Creating a model using GridSearchCV with the parameters grid 
model=GridSearchCV(svc,param_grid)

In [ ]:
# Training the model using the training data 
model.fit(x_train,y_train)

In [ ]:
# Testing the model using the testing data 
y_pred = model.predict(x_test) 

# Calculating the accuracy of the model 
accuracy = accuracy_score(y_pred, y_test) 

# Print the accuracy of the model 
print(f"The model is {accuracy*100}% accurate")